# Ancient to Modern Italian Translation with TinyLLaMA and BLOOMZ

This notebook compares two approaches for translating ancient Italian into modern Italian:

1. **TinyLLaMA (Fine-tuned locally on parallel examples)**
2. **BLOOMZ (Zero-shot / Few-shot Inference)**

---

## 🔧 Setup

In [1]:
# --- 1. Install required libraries ---
!pip install -q transformers datasets peft bitsandbytes accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 20.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.9 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 22.3 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 req

In [27]:
# --- 2. Import modules ---
import pandas as pd
import numpy as np
import torch
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling, BitsAndBytesConfig
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from sklearn.model_selection import train_test_split
import evaluate
import ast

# --- 3. Load and prepare dataset ---
df = pd.read_csv('/kaggle/input/datasets-both/dataset_concatenato.csv')[['text', 'translation']].dropna()
df = df.rename(columns={'text': 'ancient', 'translation': 'modern'})
df = df.head(200)  # for quick training/testing

def fix_list_string_to_sentence(text):
    # Try to parse string list representation to python list
    try:
        tokens = ast.literal_eval(text)
        if isinstance(tokens, list):
            return " ".join(tokens)
    except:
        pass
    return text  # fallback if parsing fails

df['modern'] = df['modern'].apply(fix_list_string_to_sentence)

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

# --- 4. Load tokenizer and model (4-bit quantized TinyLLaMA) ---
model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
model = prepare_model_for_kbit_training(model)

# --- 5. Apply LoRA ---
peft_config = LoraConfig(
    r = 16,
    lora_alpha = 64,
    lora_dropout = 0.1, target_modules=["q_proj", "v_proj"],
    bias="none", task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

# --- 6. Preprocess: tokenize and mask input (prompt-only tuning) ---
def preprocess_function(examples):
    prompts = [f"Translate from ancient to modern Italian:\nAncient: {a}\nModern: {m}{tokenizer.eos_token}" for a, m in zip(examples['ancient'], examples['modern'])]
    return tokenizer(prompts, truncation=True, padding="max_length", max_length=256)

train_ds = train_ds.map(preprocess_function, batched=True, remove_columns=train_ds.column_names)
val_ds = val_ds.map(preprocess_function, batched=True, remove_columns=val_ds.column_names)

# --- 7. Define training ---
training_args = TrainingArguments(
    output_dir="/kaggle/working/tinyllama-ft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # effective batch size = 8
    num_train_epochs=10,
    logging_steps=50,
    eval_strategy='steps',
    eval_steps=50,
    save_strategy='epoch',
    learning_rate=2e-4,
    fp16=True,
    report_to='none',
    gradient_checkpointing=True,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir="/kaggle/working/logs"
)

def clean_decoded_text(text):
        text = text.strip()
        if text.startswith("[") and text.endswith("]"):
            text = text[1:-1]  # remove brackets
        text = text.replace("'", "")  # remove quotes
        return text.strip()

bleu_metric = evaluate.load("bleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    if isinstance(preds, np.ndarray) and preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)
    if isinstance(preds, torch.Tensor):
        preds = preds.detach().cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.detach().cpu().numpy()
    
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = [pred.split("Modern:")[-1].strip() for pred in decoded_preds]
    decoded_labels = [label.split("Modern:")[-1].strip() for label in decoded_labels]

    pred_tokens = [pred.split() for pred in decoded_preds]
    label_tokens = [label.split() for label in decoded_labels]

    print("Sample prediction:", decoded_preds[0])
    print("Ground truth:", decoded_labels[0])

    result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["bleu"]}

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

# --- 8. Train the model ---
trainer.train()

# --- 9. Load test set ---
test_df = pd.read_csv('/kaggle/input/dataset-true/dataset_human_eval.csv')[['Sentence', 'HumanEval']].dropna()
test_df = test_df.rename(columns={'Sentence': 'ancient', 'HumanEval': 'modern'})

# --- 10. Inference with few-shot prompt ---
few_shot_prompt = """Translate from ancient to modern Italian:

Ancient: O Deo, co’ mi par forte non so se lo sapete, con’ v’amo di bon core;
Modern: Oddio, come mi sembra difficile non so se lo sapete quanto vi amo;

Ancient: Apparve luce, che rendé splendore, che passao per li occhi e ’l cor ferìo, ond’io ne sono a tal condizïone;
Modern: Apparve una luce splendente, che passò per gli occhi e colpì il cuore, cosicché io sono in tale condizione;

Ancient: Per lo cammino ch'è sì aspro e forte, vado cercando sol la mia salute;
Modern: Per il cammino che è così duro e difficile, vado cercando soltanto la mia salvezza;

Ancient: {input}
Modern:"""

def generate_with_tinyllama(text):
    prompt = few_shot_prompt.format(input=text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=False,     # ← deterministic (greedy) decoding
        num_beams=4,         # ← beam search helps BLEU
        temperature=0.7,        # moderate randomness
        top_p=0.9,              # nucleus sampling
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Strip everything before the final "Modern:" to get clean output
    return decoded.split("Modern:")[-1].strip()

# --- 11. Generate on test set ---
test_df['tinyllama_output'] = test_df['ancient'].apply(generate_with_tinyllama)

# --- 12. Save predictions ---
test_df[['ancient', 'modern', 'tinyllama_output']].to_csv("/kaggle/working/tinyllama_predictions.csv", index=False)

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

/tmp/ipykernel_35/3683008165.py:127: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss,Bleu
50,3.248100,2.853111,0.128961
100,2.487300,2.813025,0.128175
150,2.195500,2.849160,0.134546


Sample prediction: Io non soensovo souggare il m, terra, né non fure se l, non vedederlo sfavolre davutt'eorno, come un ferro cheantentecente;eso uscito dal fuoco;
Ground truth: Io non potei fissare il sole a lungo, ma neppure così poco da non vederlo sfavillare tutt'intorno, come un ferro incandescente appena uscito dal fuoco;
Sample prediction: Io non soensova souggare il suo, terra, né non rimure così tanto, far vedederlo sfavolre davuttaviaoreeorno, come un ferro cheantentecente;are escito dal fuoco;
Ground truth: Io non potei fissare il sole a lungo, ma neppure così poco da non vederlo sfavillare tutt'intorno, come un ferro incandescente appena uscito dal fuoco;
Sample prediction: Io non soegva souggare il suo, terra, né solo rimure così,, far vedederlo sfavolre davuttaviaoreunorno, come un ferro cheantentecente cheare escito dal fuoco;
Ground truth: Io non potei fissare il sole a lungo, ma neppure così poco da non vederlo sfavillare tutt'intorno, come un ferro incandescente appen

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [28]:
from IPython.display import display
display(test_df[['ancient', 'modern', 'tinyllama_output']].head(30))

,ancient,modern,tinyllama_output
0,"quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno","Quella guerra fu condotta bene perchè etc. Dall’altra parte, Aiace era un cavaliere franco e valoroso nelle armi, di grande statura, ma non possedeva grande saggezza.","Questa guerra è stata fatta perché Etaces era un cavaliere coraggioso e profondo in pensiero, ma non era sufficientemente dotato per l'arma"
1,"crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.","È crudele, e punisce ogni colpa come vuole la legge, e non perdona alcun cavaliere che sbagli.","cruenta, e di tutte le colpe che ho commesso pigno giudizio, come dice la legge, ed a un cavaliere perdono la pena per i miei pecchi."
2,"Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.","Ponzio Aufidiano, cavaliere romano, non fu dotato di un animo diverso.","Non d' alcuna forza d' animo fu adornato Ponzio Aufidiano, romano cavaliere;"
3,"Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.","Se questo piace a tutti e se il tempo ha bisogno di avere Pompeo come cavaliere e non come compagno, non ostacolerò più il destino.","Se questo piace a tutti e se il tempo ha bisogno di avere Pompeo come cavaliere e non come compagno, non riprenderò più i fatti."
4,"Officio di questa arte pare che sia dicere appostatamente per fare credere, fine è far credere per lo dire.",Il compito di quest’arte sembra essere quello di parlare in modo adatto per convincere; lo scopo è far credere qualcosa attraverso le parole.,"L'arte di questa disciplina sembra essere quella di dare la parola corretta per far credere, c'è fine di fare credere per l'espressione."
5,Ecco e larghi ventipiovoli caggiono delle risolute nebbie; e potresti credere che tutto il cielo cadesse nel mare,Ecco che forti piogge scendono dalle dense nebbie; potresti pensare che tutto il cielo stia cadendo in mare.,"Ecco le ciglia delle nebbie che caggiano i ventopiovoli, e potresti credere che tutto il cielo cadesse nel mare;"
6,"Però che or chi spererebbe quello che eziandio questi che non vogliono ancora credere in Cristo, già veggiono con noi, e perché nol possono negare, stridono colli denti.","Perché ora chi spererebbe ciò che anche quelli che non vogliono ancora credere in Cristo, già vedono insieme a noi, e poiché non lo possono negare, stringono i denti.","Poiché chi spera di questo che non vogliono ancora credere in Cristo, già vedono con noi, e perché non possono negare, stridono colli denti."
7,I vendimenti de' morti et le presure de' vivi fece la frode d'uno feroce re.,Le vendite dei morti e le pressioni sui vivi furono opera dell’inganno di un re crudele.,Il cammino della luce che
8,"Acciocché quegli, il quale ora per le sue gran reità è feroce e onorevole, egli d'ogni male afflitto e tormentato della impietà verso il mio padre.","Così che colui, che ora è temuto e onorato per i suoi grandi delitti, sia tormentato da ogni male a causa della sua crudeltà verso mio padre.","Altrimenti quegli uomini che ora per le sue grandi reitte è feroce e onorevole, egli d'ogni malessentimento e tormento del suo pessimismo verso il mio padre."
9,Gli uomini spessamente a stare fermi nella bugia incontra la verità.,Spesso gli uomini incontrano la verità mentre stanno fermi in una bugia.,"Apparve una luce splendente, che passò per gli occhi e colp"


In [32]:
test_dataset = pd.read_csv('/kaggle/input/datasets-both/dataset_cleaned (1).csv')[['Sentence']].dropna()
test_dataset = test_dataset.rename(columns={'Sentence': 'ancient'})

test_dataset['tinyllama_output'] = test_dataset['ancient'].apply(generate_with_tinyllama)
test_dataset[['ancient', 'tinyllama_output']].to_csv("/kaggle/working/tinyllama_test_predictions.csv", index=False)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
